In [1]:
import dai
import pandas as pd
import numpy as np
import time
from datetime import timedelta

In [2]:
ed = dai.query("SELECT DISTINCT date FROM cn_stock_bar1d WHERE date >= '2024-01-01'").df()['date'].max().strftime("%Y-%m-%d")
sd = (pd.Timestamp(ed) - timedelta(days=45)).strftime("%Y-%m-%d")

In [4]:
def calc_data(instrument, sd, ed):

    sql = """
    WITH 
    data1 AS (

        SELECT
        
            CAST(trading_day AS CHAR) AS _trading_day,
            make_date(CAST(SUBSTRING(_trading_day FROM 1 FOR 4) AS INT), CAST(SUBSTRING(_trading_day FROM 5 FOR 2) AS INT), CAST(SUBSTRING(_trading_day FROM 7 FOR 2) AS INT)) as date,
            
            instrument,
            
            SUM(ask_volume1)  OVER (PARTITION BY trading_day, instrument) AS ask_volume1_s,
            SUM(ask_volume2)  OVER (PARTITION BY trading_day, instrument) AS ask_volume2_s,
            SUM(ask_volume3)  OVER (PARTITION BY trading_day, instrument) AS ask_volume3_s,
            SUM(ask_volume4)  OVER (PARTITION BY trading_day, instrument) AS ask_volume4_s,
            SUM(ask_volume5)  OVER (PARTITION BY trading_day, instrument) AS ask_volume5_s,
            SUM(ask_volume6)  OVER (PARTITION BY trading_day, instrument) AS ask_volume6_s,
            SUM(ask_volume7)  OVER (PARTITION BY trading_day, instrument) AS ask_volume7_s,
            SUM(ask_volume8)  OVER (PARTITION BY trading_day, instrument) AS ask_volume8_s,
            SUM(ask_volume9)  OVER (PARTITION BY trading_day, instrument) AS ask_volume9_s,
            SUM(ask_volume10) OVER (PARTITION BY trading_day, instrument) AS ask_volume10_s,

            SUM(bid_volume1)  OVER (PARTITION BY trading_day, instrument) AS bid_volume1_s,
            SUM(bid_volume2)  OVER (PARTITION BY trading_day, instrument) AS bid_volume2_s,
            SUM(bid_volume3)  OVER (PARTITION BY trading_day, instrument) AS bid_volume3_s,
            SUM(bid_volume4)  OVER (PARTITION BY trading_day, instrument) AS bid_volume4_s,
            SUM(bid_volume5)  OVER (PARTITION BY trading_day, instrument) AS bid_volume5_s,
            SUM(bid_volume6)  OVER (PARTITION BY trading_day, instrument) AS bid_volume6_s,
            SUM(bid_volume7)  OVER (PARTITION BY trading_day, instrument) AS bid_volume7_s,
            SUM(bid_volume8)  OVER (PARTITION BY trading_day, instrument) AS bid_volume8_s,
            SUM(bid_volume9)  OVER (PARTITION BY trading_day, instrument) AS bid_volume9_s,
            SUM(bid_volume10) OVER (PARTITION BY trading_day, instrument) AS bid_volume10_s,

                                ask_volume1_s  AS ask_volume1_cs,
            ask_volume1_cs   +  ask_volume2_s  AS ask_volume2_cs,
            ask_volume2_cs   +  ask_volume3_s  AS ask_volume3_cs,
            ask_volume3_cs   +  ask_volume4_s  AS ask_volume4_cs,
            ask_volume4_cs   +  ask_volume5_s  AS ask_volume5_cs,
            ask_volume5_cs   +  ask_volume6_s  AS ask_volume6_cs,
            ask_volume6_cs   +  ask_volume7_s  AS ask_volume7_cs,
            ask_volume7_cs   +  ask_volume8_s  AS ask_volume8_cs,
            ask_volume8_cs   +  ask_volume9_s  AS ask_volume9_cs,
            ask_volume9_cs   +  ask_volume10_s AS ask_volume10_cs,

                                bid_volume1_s  AS bid_volume1_cs,
            bid_volume1_cs   +  bid_volume2_s  AS bid_volume2_cs,
            bid_volume2_cs   +  bid_volume3_s  AS bid_volume3_cs,
            bid_volume3_cs   +  bid_volume4_s  AS bid_volume4_cs,
            bid_volume4_cs   +  bid_volume5_s  AS bid_volume5_cs,
            bid_volume5_cs   +  bid_volume6_s  AS bid_volume6_cs,
            bid_volume6_cs   +  bid_volume7_s  AS bid_volume7_cs,
            bid_volume7_cs   +  bid_volume8_s  AS bid_volume8_cs,
            bid_volume8_cs   +  bid_volume9_s  AS bid_volume9_cs,
            bid_volume9_cs   +  bid_volume10_s AS bid_volume10_cs,

            AVG(ask_price1)  OVER (PARTITION BY trading_day, instrument) AS ask_price1_s,
            AVG(ask_price5)  OVER (PARTITION BY trading_day, instrument) AS ask_price5_s,
            AVG(ask_price6)  OVER (PARTITION BY trading_day, instrument) AS ask_price6_s,
            AVG(ask_price10) OVER (PARTITION BY trading_day, instrument) AS ask_price10_s,

            AVG(bid_price1)  OVER (PARTITION BY trading_day, instrument) AS bid_price1_s,
            AVG(bid_price5)  OVER (PARTITION BY trading_day, instrument) AS bid_price5_s,
            AVG(bid_price6)  OVER (PARTITION BY trading_day, instrument) AS bid_price6_s,
            AVG(bid_price10) OVER (PARTITION BY trading_day, instrument) AS bid_price10_s,

            (ask_price10_s - ask_price1_s) / (ask_volume10_cs - ask_volume1_cs)      AS slope_a,
            (bid_price10_s - bid_price1_s) / (bid_volume10_cs - bid_volume1_cs) * -1 AS slope_b,
            (slope_a - slope_b) AS slope_ab,

            (ask_price5_s  - ask_price1_s) / (ask_volume5_cs  - ask_volume1_cs)      AS slope_al,
            (bid_price5_s  - bid_price1_s) / (bid_volume5_cs  - bid_volume1_cs) * -1 AS slope_bl,
            (slope_al - slope_bl) / (slope_al + slope_bl) AS slope_abl,

            (ask_price10_s - ask_price6_s) / (ask_volume10_cs - ask_volume6_cs)      AS slope_ah,
            (bid_price10_s - bid_price6_s) / (bid_volume10_cs - bid_volume6_cs) * -1 AS slope_bh,
            (slope_ah - slope_bh) / (slope_ah + slope_bh) AS slope_abh,

        FROM  cn_stock_level2_snapshot
        ORDER BY date, instrument
    ),

    data2 AS (
        SELECT DISTINCT * 
        FROM data1
        ORDER BY date, instrument
    ),

    data3 AS (
        SELECT
            CAST(date AS TIMESTAMP) AS date,
            instrument,
            slope_ab  AS alpha_hf_00050001,
            slope_abh AS alpha_hf_00050002,
            slope_abl AS alpha_hf_00050003,
        FROM data2
    )

    SELECT *
    FROM data3
    ORDER BY date, instrument
    """
    data = dai.query(sql, filters = {'instrument' : [instrument], "date" : [sd+' 00:00:00.000', ed+' 23:59:59.999']}).df()
    return data